In [1]:
import sys
sys.path.append('E:/zlab')
from loader import Loader
from xtrain import XModel

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 载入数据

In [2]:
import tables as tb
import numpy as np

h5 = tb.open_file('E:/xdata/X.h5')
#h5 = tb.open_file('./X.h5')

data = h5.root.cifar10

## 划分数据集

In [3]:
from sklearn.model_selection import train_test_split

X = data.trainX[:]
y = data.trainY[:]
X_train, X_unlabel, y_train, y_unlabel = train_test_split(
    X, y, test_size=0.98, random_state=42)

batch_size = 32
trainset = Loader(batch_size, X_train, y_train, shuffle=True, name='train')
testset = Loader(
    batch_size, data.testX, data.testY, shuffle=False, name='test')
unlabelset = Loader(
    batch_size,
    X_unlabel[:10000],
    y_unlabel[:10000],
    shuffle=False,
    name='agent')

# 模型的配置

In [4]:
from mxnet.gluon import nn, model_zoo
from mxnet import nd, init


class SemiModel(nn.HybridBlock):
    def __init__(self, features, **kwargs):
        super().__init__(**kwargs)
        self.features = features
        #self.features1 = nn.HybridSequential()
        #self.features.add(nn.Dense(1024), nn.Activation('relu'))
        self.output = nn.Dense(10)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        #x= self.features1(x)
        return self.output(x)


pretrain_net = model_zoo.vision.resnet50_v2(pretrained=False)
net = SemiModel(pretrain_net.features)
net.hybridize()

# 网络预设
_net = model_zoo.vision.resnet50_v2(pretrained=True)
net.features = _net.features
net.output.initialize(init.Xavier(magnitude=0.24))

# 超参数设定
epochs = 500
learning_rate = 0.1
model = XModel(learning_rate)
ctx = model.ctx
print('training on', ctx)
net.collect_params().reset_ctx(ctx)

training on gpu(0)


In [30]:
y
        self.features_u = self.unlabel.X.reshape((-1, 32 * 32 * 3)) / 255
        self.features_l = self.train.X.reshape((-1, 32 * 32 * 3)) / 255
        self.y_agent = -np.ones_like(self.unlabel.Y)
        self.X_ = np.concatenate([self.features_l, self.features_u])
        self.y_ = np.concatenate([self.train.Y, self.y_agent])

    def get_features(self, ctx, net, data_iter):
        for x in data_iter:
            x = nd.array(x, ctx=ctx).transpose((0, 3, 1, 2))
            features = net.features(x)
            yield features

bunch = Bunch(trainset, unlabelset, batch_size=32)

In [ ]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import classification_report, accuracy_score

lbp = LabelPropagation(kernel='knn', n_jobs=-1, max_iter=100)
lbp.fit(bunch.X_, bunch.y_) 

In [35]:
y_hats = lbp.transduction_[bunch.train.Y.shape[0]:]
agency_acc = accuracy_score(bunch.unlabel.Y, y_hats)

NameError: name 'accuracy_score' is not defined

In [ ]:
def ssl(self):
        '''
        标签传播
        '''
        
        self._lbp.fit(X, y_)  # 必须将 X 缩放到 [0,1]
        self.y_hats = self._lbp.transduction_[self.train.Y.shape[0]:]
        self.agency_acc = accuracy_score(self.unlabel.Y, self.y_hats)
        print('微调前的预测精度: ', self.agency_acc)
        if np.unique(self.y_hats).size == 10:
            print('==' * 27)
            print('分类指标：')
            print(classification_report(self.unlabel.Y, self.y_hats))
            print('==' * 27)
            
            
def agent(self, net):
        '''
        net:: 神经网络
        inference::输出 net 的 features
        '''
        # 提取有标签数据的特征
        self.features_l = np.concatenate([
            f.asnumpy() for f in self.get_features(net, self.X_l)
        ])
        # 提取无标签数据的特征
        self.features_u = np.concatenate([
            f.asnumpy() for f in self.get_features(net, self.X_u)
        ])
        self.ssl()
        print('微调后的预测精度: ', self.agency_acc)